In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
import time
random.seed(0)

In [2]:
# 导入Fmnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.FashionMNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [6]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8367
SVD: 0.8285
NetD, Acc at first 0.8399


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.8065
SVD: 0.8056
NetG, Acc at first 0.8452


In [7]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 1
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 5

for epoch in range(num_steps):
    if 1==1:
        
        #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        #radius_list.append(radius)
        #print(radius)

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            #radius_list.append(radius)
            #print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()

            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        

tensor(-53.5348, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.9373, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.5018, device='cuda:0', grad_fn=<NegBackward0>), tensor(-41.0957, device='cuda:0', grad_fn=<AddBackward0>)]
99.46526550294142
tensor(-45.8039, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.5148, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.4672, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.8219, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-50.2364, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.5148, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.7495, device='cuda:0', grad_fn=<NegBackward0>), tensor(-39.9722, device='cuda:0', grad_fn=<AddBackward0>)]
5.335757674659305
tensor(-58.5598, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.8454, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.7247, device='cuda:0', grad_fn=<NegBackward0>), tensor(-45.9897, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-58.3607, device='cud

8.32097956006593
tensor(-97.1594, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.3492, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.5893, device='cuda:0', grad_fn=<NegBackward0>), tensor(-72.2210, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-96.2144, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.3492, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.2198, device='cuda:0', grad_fn=<NegBackward0>), tensor(-71.6454, device='cuda:0', grad_fn=<AddBackward0>)]
5.685076007302262
tensor(-98.7380, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.9610, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.9376, device='cuda:0', grad_fn=<NegBackward0>), tensor(-72.8395, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-98.1259, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.9610, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.6929, device='cuda:0', grad_fn=<NegBackward0>), tensor(-72.4721, device='cuda:0', grad_fn=<AddBackward0>)]
5.4715922376845
tensor(-9

5.7722375350358
tensor(-105.6946, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.9231, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.4598, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.3117, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-106.4406, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-21.9231, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.6233, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.8941, device='cuda:0', grad_fn=<AddBackward0>)]
5.615924814613502
tensor(-107.0780, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.1004, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.9963, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.9813, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-107.3101, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.1004, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7678, device='cuda:0', grad_fn=<NegBackward0>), tensor(-77.4419, device='cuda:0', grad_fn=<AddBackward0>)]
5.441161155803575
tens

5.344326205989477
tensor(-112.1346, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8967, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.9844, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.2535, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-112.2441, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8967, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.0588, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.2886, device='cuda:0', grad_fn=<AddBackward0>)]
5.973967772127399
tensor(-112.7930, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.0117, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.2569, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.5244, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-111.7516, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.0117, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.0958, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.6440, device='cuda:0', grad_fn=<AddBackward0>)]
5.763809576592772
te

5.982745853175801
tensor(-115.5458, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.8936, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1427, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.5095, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-115.2406, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.8936, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.8281, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.5190, device='cuda:0', grad_fn=<AddBackward0>)]
5.965860195030153
tensor(-116.1041, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-27.3004, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1171, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.6866, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-116.6755, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-27.3004, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3921, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.9831, device='cuda:0', grad_fn=<AddBackward0>)]
5.857967232938298
te

...

5.188268515873933
tensor(-136.2877, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2495, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.1749, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.8633, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-133.6781, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2495, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4185, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.0101, device='cuda:0', grad_fn=<AddBackward0>)]
4.991721780078178
tensor(-135.6939, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2373, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.0283, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.4283, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-135.7973, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2373, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.6481, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.9119, device='cuda:0', grad_fn=<AddBackward0>)]
5.404138439320855
te

4.775847165554779
tensor(-134.9173, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.1777, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.5578, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.1818, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-134.4510, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.1777, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.0373, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.2360, device='cuda:0', grad_fn=<AddBackward0>)]
4.687802085374222
tensor(-135.7515, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.0659, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.1344, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.5512, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-134.5559, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.0659, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.0100, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.4800, device='cuda:0', grad_fn=<AddBackward0>)]
4.553920721653676
t

5.918174343285391
tensor(-135.2558, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.0975, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5201, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.6382, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-134.4016, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.0975, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5307, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7735, device='cuda:0', grad_fn=<AddBackward0>)]
5.167480065079063
tensor(-135.4976, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.1278, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.9697, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.4000, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-132.6945, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.1278, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.9728, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.5939, device='cuda:0', grad_fn=<AddBackward0>)]
5.0623927164030915
t

5.624706488408153
tensor(-134.2478, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.1757, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3645, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7076, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-134.4191, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.1757, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8916, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.3518, device='cuda:0', grad_fn=<AddBackward0>)]
5.160611652158463
tensor(-134.9066, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2346, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8945, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7775, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-137.0540, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2346, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.6117, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.2077, device='cuda:0', grad_fn=<AddBackward0>)]
5.107308202703726
te

5.741920906934458
tensor(-134.4510, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.1841, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4300, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.8369, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-134.3602, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.1841, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4153, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7609, device='cuda:0', grad_fn=<AddBackward0>)]
6.0145003453771375
tensor(-133.9668, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2744, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.0282, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.6642, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-134.4978, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.2744, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.2688, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.9547, device='cuda:0', grad_fn=<AddBackward0>)]
5.382036272643092
t

In [8]:
radius_list

[99.46526550294142,
 5.335757674659305,
 4.9626793202252495,
 56.157269475137674,
 4.944625396214916,
 5.293334796713626,
 5.086659777634566,
 5.536800251752226,
 5.152032951282645,
 4.981625340422114,
 5.893158515731432,
 5.2156609581602185,
 5.848009191846782,
 5.2209317844907766,
 5.5387326633568055,
 6.654194278781697,
 5.454379786178018,
 8.32097956006593,
 5.685076007302262,
 5.4715922376845,
 5.784477453634504,
 6.2113488532307555,
 5.51505432744512,
 5.2360867647835425,
 5.539321810884139,
 6.0373478632081055,
 5.460426072347165,
 6.559253749084248,
 5.572715763876178,
 5.526481805476234,
 5.5132667661637385,
 5.152456795548261,
 5.885639167150233,
 5.407499819889886,
 5.7722375350358,
 5.615924814613502,
 5.441161155803575,
 5.561300117631943,
 5.644893273198948,
 5.622095788942042,
 5.374240994369011,
 5.609480362545742,
 6.011389363931392,
 6.11855306773308,
 5.666042106304526,
 6.143176898273719,
 5.059363477190161,
 5.599005917836417,
 7.378794437525651,
 5.338313685554415

In [9]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.8078
SVD: 0.7981
NetD, Acc at first 0.8579


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.804
SVD: 0.802
NetG, Acc at first 0.8376


In [10]:
torch.save(netD, 'FMNIST_NetD_changing1-mode1.pth')
torch.save(netV, 'FMNIST_NetV_changing1-mode1.pth')
torch.save(netG, 'FMNIST_NetG_changing1-mode1.pth')

#netD = torch.load('FMNIST_NetD.pth')
#netV = torch.load('FMNIST_NetV.pth')
#netG = torch.load('FMNIST_NetG.pth')

#mode = 1
#gam3 = 1

In [11]:
r_list = [0,0.1,0.2,0.5,1,2,3,4,6,8,10]
sample_num = 20

In [12]:
# sample 20 times of NetG at final radius, NetD is fixed, length 0.1,0.2,0.5,1,2,4,6,8,10,15,20
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample = []
for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[1])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample.append(count/10000)
    
print(max(Acc_results_G_sample),min(Acc_results_G_sample),np.mean(Acc_results_G_sample),np.var(Acc_results_G_sample))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.7954
SVD: 0.7876
NetG_fixD, Acc at sample 1 0.8528


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.7942
SVD: 0.7867999999999999
NetG_fixD, Acc at sample 2 0.8533


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.796
SVD: 0.7888
NetG_fixD, Acc at sample 18 0.8525


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.7933
SVD: 0.7874
NetG_fixD, Acc at sample 19 0.8525


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.7953
SVD: 0.788
NetG_fixD, Acc at sample 20 0.8528
0.8545 0.8518 0.8529899999999998 3.5690000000000167e-07


In [13]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample2 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[2])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample2.append(count/10000)
    
print(max(Acc_results_G_sample2),min(Acc_results_G_sample2),np.mean(Acc_results_G_sample2),np.var(Acc_results_G_sample2))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.7938000000000001
SVD: 0.7893
NetG_fixD, Acc at sample 1 0.8535


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.7925
SVD: 0.7863
NetG_fixD, Acc at sample 2 0.8527


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.7935
SVD: 0.7867999999999999
NetG_fixD, Acc at sample 18 0.8528


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.7939
SVD: 0.7875
NetG_fixD, Acc at sample 19 0.8526


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.7939
SVD: 0.7892
NetG_fixD, Acc at sample 20 0.8526
0.854 0.8516 0.8527799999999999 3.775999999999972e-07


In [14]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample3 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[3])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample3.append(count/10000)
    
print(max(Acc_results_G_sample3),min(Acc_results_G_sample3),np.mean(Acc_results_G_sample3),np.var(Acc_results_G_sample3))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.7958000000000001
SVD: 0.7896
NetG_fixD, Acc at sample 1 0.8519


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.7916
SVD: 0.7857000000000001
NetG_fixD, Acc at sample 2 0.8509


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.7937
SVD: 0.7883
NetG_fixD, Acc at sample 18 0.8517


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.7948
SVD: 0.7904
NetG_fixD, Acc at sample 19 0.8511


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.7928
SVD: 0.7867999999999999
NetG_fixD, Acc at sample 20 0.8528
0.8539 0.8491 0.8520749999999998 1.638875000000012e-06


In [15]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample4 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[4])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample4.append(count/10000)
    
print(max(Acc_results_G_sample4),min(Acc_results_G_sample4),np.mean(Acc_results_G_sample4),np.var(Acc_results_G_sample4))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.62s/it]


----------------------------
Train_z, Test_z
PCA: 0.8001
SVD: 0.7916
NetG_fixD, Acc at sample 1 0.8516


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.7952
SVD: 0.7894
NetG_fixD, Acc at sample 2 0.8516


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.7939
SVD: 0.7924
NetG_fixD, Acc at sample 18 0.848


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.7981
SVD: 0.7922
NetG_fixD, Acc at sample 19 0.8503


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.7919
SVD: 0.7905
NetG_fixD, Acc at sample 20 0.8511
0.8519 0.8473 0.8501099999999999 1.888899999999987e-06


In [16]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample5 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[5])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample5.append(count/10000)
    
print(max(Acc_results_G_sample5),min(Acc_results_G_sample5),np.mean(Acc_results_G_sample5),np.var(Acc_results_G_sample5))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8075
SVD: 0.8004
NetG_fixD, Acc at sample 1 0.8379


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.61s/it]


----------------------------
Train_z, Test_z
PCA: 0.8035
SVD: 0.7964
NetG_fixD, Acc at sample 2 0.8302


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8006
SVD: 0.7964
NetG_fixD, Acc at sample 18 0.8336


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8075
SVD: 0.8054
NetG_fixD, Acc at sample 19 0.8276


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.7999
SVD: 0.795
NetG_fixD, Acc at sample 20 0.8344
0.8408 0.8276 0.83558 1.0694599999999925e-05


In [17]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample6 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[6])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample6.append(count/10000)
    
print(max(Acc_results_G_sample6),min(Acc_results_G_sample6),np.mean(Acc_results_G_sample6),np.var(Acc_results_G_sample6))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8062
SVD: 0.8052
NetG_fixD, Acc at sample 1 0.7437


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8108
SVD: 0.8047
NetG_fixD, Acc at sample 2 0.6934


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8091
SVD: 0.8053
NetG_fixD, Acc at sample 18 0.6664


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8110999999999999
SVD: 0.8093
NetG_fixD, Acc at sample 19 0.7344


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.8133
SVD: 0.8083
NetG_fixD, Acc at sample 20 0.6731
0.779 0.634 0.7099150000000001 0.0013726732750000002


In [18]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample7 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[7])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample7.append(count/10000)
    
print(max(Acc_results_G_sample7),min(Acc_results_G_sample7),np.mean(Acc_results_G_sample7),np.var(Acc_results_G_sample7))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.808
SVD: 0.8037
NetG_fixD, Acc at sample 1 0.3902


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8044
SVD: 0.8044
NetG_fixD, Acc at sample 2 0.3027


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8089999999999999
SVD: 0.8081
NetG_fixD, Acc at sample 18 0.3259


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.8185
SVD: 0.8145
NetG_fixD, Acc at sample 19 0.3059


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8061
SVD: 0.8022
NetG_fixD, Acc at sample 20 0.365
0.4147 0.2151 0.333935 0.003402524275


In [19]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample8 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[8])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample8.append(count/10000)
    
print(max(Acc_results_G_sample8),min(Acc_results_G_sample8),np.mean(Acc_results_G_sample8),np.var(Acc_results_G_sample8))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.8103
SVD: 0.8116
NetG_fixD, Acc at sample 1 0.1222


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8103
SVD: 0.8008
NetG_fixD, Acc at sample 2 0.1166


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8126
SVD: 0.8056
NetG_fixD, Acc at sample 18 0.1257


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8025
SVD: 0.8025
NetG_fixD, Acc at sample 19 0.1491


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.8106
SVD: 0.8059000000000001
NetG_fixD, Acc at sample 20 0.1234
0.1643 0.1132 0.130605 0.00016547047500000003


In [20]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample9 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[9])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample9.append(count/10000)
    
print(max(Acc_results_G_sample9),min(Acc_results_G_sample9),np.mean(Acc_results_G_sample9),np.var(Acc_results_G_sample9))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.815
SVD: 0.8119000000000001
NetG_fixD, Acc at sample 1 0.1058


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.808
SVD: 0.804
NetG_fixD, Acc at sample 2 0.1171


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8088
SVD: 0.8079000000000001
NetG_fixD, Acc at sample 18 0.1001


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8076
SVD: 0.8063
NetG_fixD, Acc at sample 19 0.0978


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.8098
SVD: 0.8034
NetG_fixD, Acc at sample 20 0.1057
0.1236 0.0879 0.10887500000000001 9.477387499999997e-05


In [21]:
# sample 50 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample10 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[10])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample10.append(count/10000)
    
print(max(Acc_results_G_sample10),min(Acc_results_G_sample10),np.mean(Acc_results_G_sample10),np.var(Acc_results_G_sample10))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8089999999999999
SVD: 0.8049
NetG_fixD, Acc at sample 1 0.1114


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.8047
SVD: 0.8038
NetG_fixD, Acc at sample 2 0.1005


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.8072
SVD: 0.8076
NetG_fixD, Acc at sample 18 0.1124


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.8099000000000001
SVD: 0.8056
NetG_fixD, Acc at sample 19 0.0803


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.8107
SVD: 0.8039000000000001
NetG_fixD, Acc at sample 20 0.1073
0.1146 0.0763 0.10117000000000001 0.00010806709999999996
